### Test time cost on IBMQ

In [ ]:
import argparse
import os
import sys
import yaml

import lightning as L
import pandas as pd
import pennylane as qml
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.accounts import AccountAlreadyExistsError
from qiskit_ibm_runtime.exceptions import IBMNotAuthorizedError
import torch

from source.models.qcgnn import QCGNN_IX
from source.utils.gmail import send_email

In [ ]:
if 'ipykernel' in sys.argv[0]:
    backend = ''
else:
    parser = argparse.ArgumentParser(description='Process some inputs.')
    parser.add_argument('--backend', type=str, default=None, help='Backend (default: None)')
    parser_args = parser.parse_args()
    backend = parser_args.backend

try:
    print('Test IBMQ Authentication...')
    service = QiskitRuntimeService(instance='ibm-q-hub-ntu/ntu-internal/default')
    ibmq_backends = service.backends()
    for ibmq_backend in ibmq_backends:
        num_jobs =  ibmq_backend.status().pending_jobs
        print(f"IBMQ backend {ibmq_backend} has {num_jobs} pending jobs.")
except (AccountAlreadyExistsError, IBMNotAuthorizedError):
    # If 401 error: https://docs.quantum.ibm.com/announcements/service-alerts/2024-06-05-api-token-refresh
    token = input('Enter IBMQ token: ')
    QiskitRuntimeService.save_account(
        channel='ibm_quantum',
        token=token,
        set_as_default=True,
        overwrite=True
    )

In [ ]:
def run_on_ibmq(backend:str, n_I: int, n_R: int, n_L: int) -> list[dict]:

    print(f"\n----- n_R = {n_R} | n_I = {n_I} | n_L = {n_L} -----")

    result_list: list[dict] = []

    for rnd_seed in range(10):

        L.seed_everything(rnd_seed)
        x = torch.rand(2 ** n_I, 3 * n_Q)

        model = QCGNN_IX(
            num_ir_qubits=n_I,
            num_nr_qubits=n_Q,
            num_layers=n_L,
            num_reupload=n_R,
            num_rotation=3,
            vqc_ansatz=qml.StronglyEntanglingLayers,
            qdevice='qiskit.ibmq',
            qbackend=backend,
        )

        _ = model(x)
        job_id = model.qml_device._current_job.job_id()

        service = QiskitRuntimeService(instance='ibm-q-hub-ntu/ntu-internal/default')
        job = service.job(job_id)
        job_runtime = job.result().time_taken

        result_list.append({
            'n_Q': n_Q,
            'n_I': n_I,
            'n_R': n_R,
            'n_L': n_L,
            'job_id': job_id,
            'job_runtime': job_runtime,
            'backend': backend,
            'rnd_seed': rnd_seed,
        })

        print(f"* Job id: {job_id} | seed = {rnd_seed} -> time = {job_runtime:.3f}")

    return result_list

n_Q = 100
n_I_list = [1, 2, 3]
n_R_list = [0, 10]
n_L_list = [0, 10]

result_list = []

try:
    status = 'Success'
    message = ''

    for n_R in n_R_list:
        for n_I in n_I_list:
            if n_R == 0:
                result_list += run_on_ibmq(backend, n_R=0, n_I=n_I, n_L=0)
            else:
                for n_L in n_L_list:
                    result_list += run_on_ibmq(backend, n_R=n_R, n_I=n_I, n_L=n_L)

    with open(f"configs/ibmq.yaml", 'r') as file:
        ibmq_config = yaml.safe_load(file)
        save_dir = os.path.join(ibmq_config['Result']['output_dir'], 'time_complexity')
        os.makedirs(save_dir, exist_ok=True)

    df = pd.DataFrame(result_list)
    df.to_csv(os.path.join(save_dir, f"{backend}_Q{n_Q}.csv"), index=False)

except Exception as e:
    status = 'Failed'
    message = e

finally:
    send_email(
        subject=f"IBMQ_time_complexity-{backend}: {status}",
        message=message,
        config={},
    )